# Video Game Review Analysis: Critics vs. Users
### A Statistical Study of Metacritic, IGN, and OpenCritic Trends

This notebook explores whether professional game critics and general players share the same perspective on game quality. We look for "review inflation" over time and identify games with the largest sentiment gaps.

## 1. Data Loading
The loading of the dataset and initial setup.

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_rel, levene, pearsonr
import matplotlib.pyplot as plt
import seaborn as sns

# Setting visualization style
sns.set_theme(style="whitegrid")

try:
    ign = pd.read_csv('Databases/IGN_data.csv')
    meta = pd.read_csv('metacritic_pc_games.csv')
    oc = pd.read_csv('Opencritic_dataset.csv')
except FileNotFoundError:
    ign = pd.read_csv('IGN_data.csv')
    meta = pd.read_csv('metacritic_pc_games.csv')
    oc = pd.read_csv('Opencritic_dataset.csv')

## 2. Preprocessing & Normalization
We standardize the column names and scale all scores to a **0-100** range to ensure fair comparisons between sources like IGN (0-10) and Metacritic (0-100).

In [ ]:
for df in [ign, meta, oc]:
    df.columns = df.columns.str.strip()

# Standardizing Column Names
meta.rename(columns={'Game Title':'game', 'Overall Metascore':'critic_score', 
                     'Overall User Rating':'user_score', 'Game Release Date':'date'}, inplace=True)
oc.rename(columns={'Title':'game', 'Score':'critic_score_oc', 'Release Date':'date_oc'}, inplace=True)
ign.rename(columns={'game':'game', 'score':'critic_score_ign', 'released_date':'date_ign'}, inplace=True)

# Date Conversion
meta['date'] = pd.to_datetime(meta['date'], errors='coerce')
oc['date_oc'] = pd.to_datetime(oc['date_oc'], errors='coerce')
ign['date_ign'] = pd.to_datetime(ign['date_ign'], errors='coerce')

# Merging
merged = meta.merge(oc, on='game', how='outer').merge(ign, on='game', how='outer')

# Scaling 0-10 sources to 0-100
for col in ['critic_score', 'user_score', 'critic_score_oc', 'critic_score_ign']:
    merged[col] = pd.to_numeric(merged[col], errors='coerce')

merged['user_score'] = merged['user_score'] * 10
merged['critic_score_ign'] = merged['critic_score_ign'] * 10

# Year consolidation
merged['year'] = merged['date'].dt.year.fillna(merged['date_oc'].dt.year).fillna(merged['date_ign'].dt.year)
merged_clean = merged.dropna(subset=['year']).copy()

## 3. Hypothesis Testing
We apply statistical tests to validate our observations:
* **Pearson Correlation**: To see if scores are rising/falling over the years.
* **Paired T-Test**: To determine if the average difference between Critics and Users is statistically significant.
* **Levene's Test**: To compare the variance (spread) of scores.

In [ ]:
test_results = []

# H1: Metacritic Trend
df_h1 = merged_clean.dropna(subset=['critic_score', 'year'])
corr_meta, p_meta = pearsonr(df_h1['year'], df_h1['critic_score'])
test_results.append({'Hypothesis': 'H1: Critics vs Year', 'Stat': corr_meta, 'P-Value': p_meta})

# H2: Critic vs User Mean (Paired)
df_h2 = merged_clean.dropna(subset=['critic_score', 'user_score'])
t_stat, p_val_h2 = ttest_rel(df_h2['critic_score'], df_h2['user_score'])
test_results.append({'Hypothesis': 'H2: Critic vs User Mean', 'Stat': t_stat, 'P-Value': p_val_h2})

# H3: IGN Trend
df_h3 = merged_clean.dropna(subset=['critic_score_ign', 'year'])
corr_ign, p_ign = pearsonr(df_h3['year'], df_h3['critic_score_ign'])
test_results.append({'Hypothesis': 'H3: IGN vs Year', 'Stat': corr_ign, 'P-Value': p_ign})

# H4: Variance Comparison
stat_var, p_val_var = levene(df_h2['user_score'], df_h2['critic_score'])
test_results.append({'Hypothesis': 'H4: Variance (User vs Critic)', 'Stat': stat_var, 'P-Value': p_val_var})

summary_df = pd.DataFrame(test_results)
summary_df['Significant?'] = summary_df['P-Value'] < 0.05
summary_df

## 4. Visual Analysis
Visualizing the distributions and trends discussed above.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Trend Over Time
yearly_trends = merged_clean.groupby('year')[['critic_score', 'critic_score_ign']].mean()
sns.lineplot(data=yearly_trends, ax=axes[0,0], palette="magma", linewidth=2.5)
axes[0,0].set_title('Average Review Scores Over Time (0-100 Scale)')

# Distributions
sns.kdeplot(merged_clean['critic_score'], label='Critics (Meta)', fill=True, ax=axes[0,1])
sns.kdeplot(merged_clean['user_score'], label='Users (Meta)', fill=True, ax=axes[0,1])
axes[0,1].set_title('Density of Review Scores')
axes[0,1].legend()

# Boxplot for Spread
df_melted = df_h2.melt(id_vars=['game'], value_vars=['critic_score', 'user_score'], 
                        var_name='Type', value_name='Score')
sns.boxplot(x='Type', y='Score', data=df_melted, ax=axes[1,1], palette="Set2")
axes[1,1].set_title('Spread of Scores: Critics vs. Users')

plt.tight_layout()
plt.show()

## 5. Identifying Controversial Titles
We define controversy as the absolute difference between Critic scores and User scores.

In [ ]:
def top_discrepancies_all_time(df, n=5):
    paired = df.dropna(subset=['critic_score', 'user_score', 'game']).groupby('game', as_index=False).agg({
        'critic_score': 'mean', 
        'user_score': 'mean'
    })
    paired['score_diff'] = paired['critic_score'] - paired['user_score']
    paired['abs_diff'] = paired['score_diff'].abs()
    return paired.sort_values('abs_diff', ascending=False).head(n)

print("TOP 5 MOST CONTROVERSIAL GAMES OF ALL TIME")
top_discrepancies_all_time(merged_clean)